In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, lil_matrix
import numpy as np
from copy import deepcopy

# Exponential diffusion kernel

In [2]:
vertices = pd.read_csv('vertices.csv') ## вершины ребер, то есть юр.лица
edges = pd.read_csv('edges.csv') ## ребра графа, то есть транзакции между юр.лицами
test = pd.read_csv('ids.csv') ## юр.лица, для которых необходимо восстановить ребра

In [12]:
def conj_matrix(df):
    df = df[['id_1', 'id_2']]
    
    df_rev = pd.DataFrame()
    df_rev['id_1'] = df.id_2
    df_rev['id_2'] = df.id_1
    
    df_final = df.append(df_rev, ignore_index=True, sort=False)
    
    df_final['status'] = 1
    
    return csr_matrix((df_final['status'], (df_final.id_1, df_final.id_2)))

In [13]:
conj_mat = conj_matrix(edges)
conj_mat_final = lil_matrix(conj_mat.shape)

In [24]:
def n_comm_neigh(mat, test, n=1):
    final = lil_matrix((test.shape[0], mat.shape[1]))
    
    for time in range(n):
        if time == 0:
            for v in range(test.shape[0]):
                final[v, :] = mat[test.id[v],:] @ mat
        else:
            for v in range(test.shape[0]):
                final[v, :] = mat[test.id[v],:] @ final
    
    final = pd.DataFrame(final.toarray())
    final.index = test.id
    return final

In [25]:
%%time
n_comm_neigh(conj_mat, test)

Wall time: 20.1 s


,0,1,2,3,4,5,6,7,8,9,...,1534740,1534741,1534742,1534743,1534744,1534745,1534746,1534747,1534748,1534749
id,,,,,,,,,,,,,,,,,,,,,
524354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1142564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83777,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1526265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
(conj_mat.T - conj_mat).sum()

0

In [8]:
def exponential_matrix(mat, test, n=5):
    
    import math
    
    def exp_mat(mat, v, n):
        
        res = mat[v]
        
        mat_n_stage = mat[v]
        
        for i in range(1, n + 1):
            mat_n_stage = (mat_n_stage @ mat)
            res += mat_n_stage / (i + 1)
        
        return res
    
    scores = np.array([])
    start = np.array([])
    end = np.array([])
    
    for v in test.id:
        
        res = exp_mat(mat, v, n).toarray().flatten()
        starts = np.array([i for i in range(len(res))])
        ends = np.array([v for _ in range(len(res))])
        
        ind = np.argwhere(res > 0).flatten()
        res, starts, ends = np.where(res > 0)[0], starts[ind], ends[ind]
                
        scores = np.concatenate((scores, res), axis=None)
        start = np.concatenate((start, starts), axis=None)
        end = np.concatenate((end, ends), axis=None)
    
    scores = np.sort(scores)
    ind = np.argsort(scores)
    start, end = start[ind], end[ind]
    
    data = pd.DataFrame()
    data['id_1'] = start
    data['id_2'] = end
    data['score'] = scores
    data['status'] = [mat[i, j] for i, j in zip(start, end)]
    
    data = data.astype(int).sort_values(by='score', ascending=False).reset_index()
    
    return data[data.status != 1][['id_1', 'id_2']].iloc[:100000, :]

In [9]:
%%time
res = exponential_matrix(conj_mat, test, n=1)

Wall time: 2min 47s


,id_1,id_2
0,1534602,1244877
1,1534582,1244877
3,1534320,1244877
4,1534214,1244877
5,1533969,1244877
...,...,...
102534,123687,912470
102535,123551,912470
102536,123535,912470
102537,123530,912470


In [ ]:
res.to_csv('ExpKernel.csv', index=False)